# Push the model's prediction to Kaggle

### Load the model and the data


In [8]:
import pickle
import sys, os
sys.path.append(os.path.abspath("."))
from dmsensei import DataModule, create_model, metrics
import numpy as np
import pandas as pd
from lightning.pytorch import Trainer
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from dmsensei.core import DataModule

model = pickle.load(open('model.pkl','rb'))
trainer = Trainer()
import torch


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


### Run the prediction

In [ ]:
prediction = trainer.predict(
        model,
        datamodule=DataModule(
            name=["ribo-test"],
            data='sequence',
            force_download=False,
            batch_size=256,
            num_workers=1,
            train_split=0,
            valid_split=0,
            predict_split=1.,
            overfit_mode=False,
        ),
    )

### Format the prediction and save it locally

In [ ]:
import json

def format_to_ribonanza(prediction):
    
    # load data (EDIT THIS)
    data = json.load(open('/root/DMSensei/scripts/data/input_files/ribo-test/data.json'))
    
    # reformat into individual sequences
    arr = [p for batch in prediction for p in batch]
    
    # remove padding
    arr = [a[:s] for a,s in zip(arr, [len(d['sequence']) for d in data.values()])] 
    
    # stack into dataframe
    prediction = np.vstack(arr)
    return pd.DataFrame(prediction, columns=["reactivity_DMS_MaP", "reactivity_2A3_MaP"]).reset_index().rename(columns={"index": "id"})

format_to_ribonanza(prediction).to_csv('submission.csv', index=False)

### Push the prediction to Kaggle

Note: setup your Kaggle authentification first:
1. download your Kaggle API keys `kaggle.json` here: https://www.kaggle.com/settings/account
2. save it to `~/.kaggle/kaggle.json`:

    ```bash
    mv ~/Downloads/kaggle.json ~/.kaggle/kaggle.json
    ```
3. push your results to kaggle using the following line

In [ ]:
os.system('kaggle competitions submit -c stanford-ribonanza-rna-folding -f submission.csv -m "test commit"')